In [1]:
from neomaril_codex.training import NeomarilTrainingClient

In [2]:
client = NeomarilTrainingClient()
client

2024-04-29 16:28:41.711 | INFO     | neomaril_codex.base:__init__:20 - Loading .env
2024-04-29 16:28:45.066 | INFO     | neomaril_codex.base:__init__:31 - Successfully connected to Neomaril


NeomarilTrainingClient(url="http://localhost:7070/api", version="eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6IlFnc0JWQ0I5WFc0V1YtSkVCVkJiZyJ9.eyJodHRwczovL25lb21hcmlsLmRhdGFyaXNrLm5ldC9uZW9tYXJpbC1ncm91cCI6ImRhdGFyaXNrIiwiaHR0cHM6Ly9uZW9tYXJpbC5kYXRhcmlzay5uZXQvZW1haWwiOiJndWlsaGVybWUuY2FyZG9zb0BkYXRhcmlzay5pbyIsImh0dHBzOi8vbmVvbWFyaWwuZGF0YXJpc2submV0L3RlbmFudCI6ImRhdGFyaXNrIiwiaHR0cHM6Ly9uZW9tYXJpbC5kYXRhcmlzay5uZXQvdXNlci1hY3RpdmUiOnRydWUsImlzcyI6Imh0dHBzOi8vZGV2LW1rM283bGF6eGxlMzBod3EudXMuYXV0aDAuY29tLyIsInN1YiI6ImF1dGgwfDY1YTE1NTA0ZDRiOGYwZDdiNDBlOGE0YiIsImF1ZCI6WyJodHRwczovL2Rldi1tazNvN2xhenhsZTMwaHdxLnVzLmF1dGgwLmNvbS9hcGkvdjIvIiwiaHR0cHM6Ly9kZXYtbWszbzdsYXp4bGUzMGh3cS51cy5hdXRoMC5jb20vdXNlcmluZm8iXSwiaWF0IjoxNzE0NDE4OTIzLCJleHAiOjE3MTQ0Mjk3MjMsInNjb3BlIjoib3BlbmlkIHByb2ZpbGUgZW1haWwgYWRkcmVzcyBwaG9uZSByZWFkOmN1cnJlbnRfdXNlciB1cGRhdGU6Y3VycmVudF91c2VyX21ldGFkYXRhIGRlbGV0ZTpjdXJyZW50X3VzZXJfbWV0YWRhdGEgY3JlYXRlOmN1cnJlbnRfdXNlcl9tZXRhZGF0YSBjcmVhdGU6Y3VycmVudF91c2VyX2RldmljZV9jcm

In [5]:
training = client.create_training_experiment(
    experiment_name='Teste',
    model_type='Classification',
    group='groupname'
)

2024-04-29 16:30:01.602 | INFO     | neomaril_codex.training:create_training_experiment:1335 - New Training 'Teste' inserted.
2024-04-29 16:30:01.604 | INFO     | neomaril_codex.base:__init__:20 - Loading .env
2024-04-29 16:30:01.607 | INFO     | neomaril_codex.base:__init__:31 - Successfully connected to Neomaril


In [8]:
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [9]:
base_path = './samples/train/'

In [10]:
df = pd.read_csv(base_path+"/dados.csv")

In [11]:
X = df.drop(columns=['target'])
y = df[["target"]]

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(df["mean_radius"], df["mean_texture"])

# Configurar o título do gráfico
plt.title("Relação entre mean_radius e mean_texture")

# Configurar os rótulos dos eixos
plt.xlabel("mean_radius")
plt.ylabel("mean_texture")

fig = plt.gcf()

# Exibir o gráfico
plt.show()


In [12]:
pipe = make_pipeline(SimpleImputer(), LGBMClassifier(force_col_wise=True))

In [13]:
pipe.fit(X, y)

[LightGBM] [Info] Number of positive: 357, number of negative: 212
[LightGBM] [Info] Total Bins 5676
[LightGBM] [Info] Number of data points in the train set: 569, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.627417 -> initscore=0.521150
[LightGBM] [Info] Start training from score 0.521150
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

/home/kardoso/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kardoso/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('lgbmclassifier', LGBMClassifier(force_col_wise=True))])

In [16]:
with training.log_train(name='Teste 1', X_train=X, y_train=y) as logger:
    logger.save_model(pipe)
    
    model_output = pd.DataFrame({"pred": pipe.predict(X), "proba": pipe.predict_proba(X)[:,1]})
    
    logger.save_model_output(model_output)

    auc = cross_val_score(pipe, X, y, cv=5, scoring="roc_auc")
    f_score = cross_val_score(pipe, X, y, cv=5, scoring="f1")
    logger.save_metric(name='auc', value=auc.mean())
    logger.save_metric(name='f1_score', value=f_score.mean())

    logger.set_python_version('3.10')

/home/kardoso/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kardoso/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/kardoso/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kardoso/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1

[LightGBM] [Info] Number of positive: 286, number of negative: 169
[LightGBM] [Info] Total Bins 4542
[LightGBM] [Info] Number of data points in the train set: 455, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.628571 -> initscore=0.526093
[LightGBM] [Info] Start training from score 0.526093
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

/home/kardoso/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kardoso/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/kardoso/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kardoso/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1

[LightGBM] [Info] Number of positive: 286, number of negative: 170
[LightGBM] [Info] Total Bins 4572
[LightGBM] [Info] Number of data points in the train set: 456, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.627193 -> initscore=0.520193
[LightGBM] [Info] Start training from score 0.520193
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

/home/kardoso/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kardoso/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/kardoso/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kardoso/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1

[LightGBM] [Info] Number of positive: 285, number of negative: 170
[LightGBM] [Info] Total Bins 4536
[LightGBM] [Info] Number of data points in the train set: 455, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.626374 -> initscore=0.516691
[LightGBM] [Info] Start training from score 0.516691
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

2024-04-29 16:32:01.444 | INFO     | neomaril_codex.training:__upload_training:915 - {"ExecutionId":1,"Message":"External training files have been partially uploaded, in order to use all features, please upload all training files! Use the id \u00271\u0027 to execute the train experiment."}
2024-04-29 16:32:02.083 | INFO     | neomaril_codex.training:__execute_training:939 - Model training starting - Hash: T855f4add2684357aaa9cdee91c0d940cf33c15ed8dc43a4ab383778a42b1e0b
2024-04-29 16:32:02.114 | INFO     | neomaril_codex.base:__init__:20 - Loading .env
2024-04-29 16:32:02.119 | INFO     | neomaril_codex.base:__init__:31 - Successfully connected to Neomaril
2024-04-29 16:32:02.121 | INFO     | neomaril_codex.base:__init__:279 - Loading .env
